In [ ]:
import Pkg
Pkg.activate(".")

# not for the faint of heart!
# Pkg.update()

pkgs = [
"ProgressMeter",
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

Pkg.develop(path="../../..")
import Mycelia

In [ ]:
function trim_galore(;outdir="", identifier="")
    
    trim_galore_dir = joinpath(outdir, "trim_galore")
    
    forward_reads = joinpath(outdir, "$(identifier)_1.fastq.gz")
    reverse_reads = joinpath(outdir, "$(identifier)_2.fastq.gz")
    
    trimmed_forward_reads = joinpath(trim_galore_dir, "$(identifier)_1_val_1.fq.gz")
    trimmed_reverse_reads = joinpath(trim_galore_dir, "$(identifier)_2_val_2.fq.gz")
    
    # mamba install -c bioconda trim_galore
    if !isfile(trimmed_forward_reads) && !isfile(trimmed_reverse_reads)
        cmd = `conda run -n viral-pangenome-discovery trim_galore --suppress_warn --cores $(min(Sys.CPU_THREADS, 4)) --output_dir $(trim_galore_dir) --paired $(forward_reads) $(reverse_reads)`
        run(cmd)
    else
        @info "$(trimmed_forward_reads) & $(trimmed_reverse_reads) already present"
    end
end

In [ ]:
srr_list = "$(dirname(pwd()))/metadata/exposome/SraAccList.txt"
srr_identifiers = readlines(srr_list)

In [ ]:
srr_identifiers_to_download = filter(srr_identifier -> !isdir("$(dirname(pwd()))/data/SRA/$(srr_identifier)") || !("kraken" in readdir("$(dirname(pwd()))/data/SRA/$(srr_identifier)")), srr_identifiers)

In [ ]:
done = false
while !done
    ProgressMeter.@showprogress for srr_identifier in srr_identifiers_to_download
        sample_outdir = "$(dirname(pwd()))/data/SRA/$(srr_identifier)"
        download_and_filter_reads(outdir=sample_outdir, srr_identifier=srr_identifier)
    end
    done = true
end
@show done